In [2]:
import json
import pandas as pd
import plotly.express as px
import re
# import plotly.io as pio
# pio.renderers.default = 'browser'

Import the geo.json file of European countries

In [117]:
#geojson file
data = json.load(open('countries.geojson','r'))

#population data
stats = pd.read_csv('population_eu.csv')

#iso code to country name (needed to relate geojson to population data)
iso_trans = pd.read_html('https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2#Decoding_table')[2]

#a list of all 27 countries in the European Union
tables = pd.read_html('https://worldpopulationreview.com/country-rankings/european-union-countries')[0]
list_eu = [item for sublist in table.values for item in sublist]

Add country name and keep only European countries in the population dataset

In [118]:
#fixing iso_trans
iso_trans = iso_trans.rename(columns = {'Country name (using title case)': 'Name'})
iso_trans['Code'] = iso_trans['Code'].apply(lambda x:str(x).lower())

#adding name and filtering for european
eu_countries = []
country_names = []

for country in data['features']:
    if len(iso_trans[iso_trans['Code'] == country['properties']['cca2']]) != 0:
        name = iso_trans[iso_trans['Code'] == country['properties']['cca2']].values[:,1][0]
        country['properties']['id'] = name
        country['id'] = name
    
    if country['properties'].get('id',0) in list(stats['country_name']):
        eu_countries.append(country)
        country_names.append(country['properties']['id']) 


In [119]:
#apparently my geojson file does not contain three european countries so I remove these from the population dataset
country_names_s = set(country_names)
data_names = set(list(stats['country_name']))

list(data_names & country_names_s)
stats = stats[stats['country_name'].apply(lambda x: x in list(data_names & country_names_s))]


In [120]:
#add id column to population dataset and do some fixing
# stats.rename(columns = {'country_name': 'id'}, inplace = True)
stats.pop('Unnamed: 0')
stats['population'] = stats['population'].apply(lambda x:int(x.replace(',','')))

stats.head()

,country_name,Continent,region,local_name,capital,area,population,population_per_sq_km,male_life_expectancy,female_life_expectancy,birth_rate,death_rate
0,Austria,Europe,Western Europe,Österreich,Vienna,"83,879 km²",8917000,106.3,78.9,83.6,9.4,10.3
1,Belgium,Europe,Western Europe,België / Belgique,Brussels,"30,530 km²",11544000,378.1,78.6,83.1,9.9,11.0
2,France,Europe,Western Europe,France,Paris,"549,087 km²",67380000,122.7,79.2,85.3,10.9,9.9
3,Germany,Europe,Western Europe,Deutschland,Berlin,"357,580 km²",83161000,232.6,78.6,83.4,9.3,11.9
4,Liechtenstein,Europe,Western Europe,Liechtenstein,Vaduz,161 km²,38137,237.6,80.1,83.6,9.1,8.2


In [142]:
fig = px.choropleth(stats,locations = 'country_name', geojson = eu_countries, 
                    color = 'population', scope = 'europe',
                   color_continuous_scale = 'Viridis',
                   locationmode = 'country names')

fig.show()
